# 使用Yolo检测电动车

In [ ]:
import cv2
import numpy
from PIL import Image, ImageDraw, ImageFont
def create_video_writer(video_cap, output_filename):

    # grab the width, height, and fps of the frames in the video stream.
    # frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    # frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_width=512
    frame_height=512
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter(output_filename, fourcc, fps,
                             (frame_width, frame_height))

    return writer

def cv2ImgAddText(img, text, left, top, textColor=(0, 255, 0), textSize=20):
    if (isinstance(img, numpy.ndarray)):  #判断是否OpenCV图片类型
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    fontText = ImageFont.truetype(
        "font/Deng.ttf", textSize, encoding="utf-8")
    draw.text((left, top), text, textColor, font=fontText)
    return cv2.cvtColor(numpy.asarray(img), cv2.COLOR_RGB2BGR)


In [ ]:
import datetime
from ultralytics import YOLO
import cv2

# define some constants
CONFIDENCE_THRESHOLD = 0.4
GREEN = (0, 255, 0)
# cap = cv2.VideoCapture(0)
# initialize the video capture object
video_cap = cv2.VideoCapture("test5.mp4")
# initialize the video writer object
writer = create_video_writer(video_cap, "output2.mp4")

# load the pre-trained YOLOv8n model
model = YOLO("models/best.pt")

while True:
    # start time to compute the fps

    ret, frame = video_cap.read()
    
    # if there are no more frames to process, break out of the loop
    if not ret:
        break
    if frame is None:
        continue
    frame = cv2.resize(frame,(512,512))
    # h = 10
    # # templateWindowSize用于计算权重的模板补丁的像素大小，为奇数，默认7
    # templateWindowSize = 3
    # # searchWindowSize窗口的像素大小，用于计算给定像素的加权平均值，为奇数，默认21
    # searchWindowSize = 21
    # frame = cv2.fastNlMeansDenoisingColored(frame, None, h, h, templateWindowSize, searchWindowSize)
    # run the YOLO model on the frame
    detections = model(frame)[0]
    
        # loop over the detections
    for data in detections.boxes.data.tolist():
        # extract the confidence (i.e., probability) associated with the detection
        confidence = data[4]

        # filter out weak detections by ensuring the 
        # confidence is greater than the minimum confidence
        if float(confidence) < CONFIDENCE_THRESHOLD:
            continue

        # if the confidence is greater than the minimum confidence,
        # draw the bounding box on the frame
        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        detected = True
        cv2.rectangle(frame, (xmin, ymin) , (xmax, ymax), GREEN, 2)
        frame = cv2ImgAddText(frame,f'已检测到电动车: {int(confidence*1000)/10}%',50,400,(255,255,0))


        # frame = cv2.putText(frame, f'已检测到电动车{confidence}', (50,150), cv2.FONT_HERSHEY_SIMPLEX, 
        #            0.3, (255,255,0), 1, cv2.LINE_AA)
    # show the frame to our screen
    if len(detections.boxes.data.tolist()) == 0:
        frame = cv2ImgAddText(frame,f'未检测到电动车, 可能位置未摆好',50,400,(255,255,0))
    cv2.imshow("Frame", frame)
    writer.write(frame)
    writer.write(frame)
    writer.write(frame)

    if cv2.waitKey(1) == ord("q"):
        break
video_cap.release()
writer.release()
cv2.destroyAllWindows()